In [1]:
import os
if '_cwd_set' not in locals(): locals()['_cwd_set'] = os.chdir(os.path.dirname(os.getcwd()))
import omnifig as fig
fig.initialize()
from src.jimports import *
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, precision_recall_curve
# for all stats at once
from sklearn.metrics import precision_recall_fscore_support
from src.util import set_default_device, repo_root, data_root
from src.dataset import RawCOCO, SimpleCOCO, RawCOCOCaptions, COCOCaptions, COCO, MNIST
device = set_default_device();

Selecting CUDA device 2 with 47642 MiB free memory and 6% utilization


In [2]:
# fullddata = COCO(eval_split=-0.1)
syscfg = fig.create_config('h/ws2')
syscfg.silent=True
fullddata = COCO(eval_split=None, split='val', dataroot=syscfg.pull('dataroot'))
fullddata.prepare(device=device);
fiids = fullddata.get_image_id(np.arange(fullddata.size))
idmap = {}
for i, im in enumerate(fiids.tolist()):
    idmap.setdefault(im, []).append(i)
picks = np.array([options[0] for options in idmap.values()])
fullddata, len(picks)

(COCO[25014](index, text_features, label, image_id, caption_id, caption, image, image_features),
 5000)

In [3]:
loc = '/data/felix/cache/checkpoints/vae128_coco_20250116_222421/ckpt_100000'
loc = '/data/felix/cache/checkpoints/ae128_coco_20250114_230254/ckpt_020000/'
loc = '/data/felix/cache/checkpoints/sae128_coco_20250118_213955/ckpt_100000'
loc = '/ssd/felix/cache/checkpoints/sae512_coco_20250124_164611/ckpt_100000/'
loc = Path(loc)

In [9]:
# cfg = fig.create_config(*'norm a/wide m/ced-man d/coco-img'.split())
cfg = fig.create_config(str(loc / 'config.yaml'))

In [ ]:
trainer = cfg.pull('trainer', silent=True)
model = trainer.model
traindataset = cfg.pull('dataset', silent=True)
valdataset = traindataset.as_eval()
traindataset.prepare(device=device);
valdataset.prepare(device=device)
system = Structured(traindataset, *trainer.gadgetry())
system.mechanize() # sync for gears and spaces
mech = system.mechanics()
model.prepare(device=device);

for p in model.parameters():
    p.requires_grad = False
model.load_checkpoint(path=loc.joinpath('model'));

/home/felix/workspace/clones/omni-learn/omnilearn/machines.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path, map_location='cpu')


In [14]:
trainable_params = sum(p.numel() for p in trainer.intervention.module.parameters())
trainable_params

1836544

In [8]:
# val_img_ids = valdataset.get_image_id(np.arange(valdataset.size))
# safe_ids = set(fiids.tolist()).intersection(set(val_img_ids.tolist()))
# len(val_img_ids)

In [9]:
# val_img_ids = valdataset.get_image_id(np.arange(valdataset.size))
# fiids = fullddata.get_image_id(np.arange(fullddata.size))
# fiid_map = {fiid: i for i, fiid in enumerate(fiids)}
# # inds = np.array([fiid_map[fiid] for fiid in safe_ids])
# # inds.sort()
# # save_json(inds.tolist(), 'save_inds.json')
# inds = load_json('save_inds.json')
# inds = np.array(inds)
# len(inds), inds[:5]

In [10]:
valfulldata = COCO(eval_split=-0.1)
# fullddata = COCO(eval_split=None, split='val')
valfulldata.prepare(device=device);
fiids = valfulldata.get_image_id(np.arange(valfulldata.size))
valmap = {}
for i, im in enumerate(fiids.tolist()):
    valmap.setdefault(im, []).append(i)
val_img_ids = set(valdataset.get_image_id(np.arange(valdataset.size)).tolist())
valpicks = np.array([options[0] for key, options in valmap.items() if key in val_img_ids])
len(valpicks)

4135

In [11]:
inds = picks
batch = Context(fullddata, DictGadget({'index': inds, 'size': len(inds)}))
inds = valpicks
valbatch = Context(valfulldata, DictGadget({'index': inds, 'size': len(inds)}))
valbatch

Context({index}, {text_features}, {label}, {image_id}, {caption_id}, {caption}, {image}, {image_features}, {size})

In [15]:
valemb = model.encode(valbatch['image_features'])
lbls = valbatch['label']
valemb.shape, lbls.shape
# lbl = 40
# lbls[:, lbl].sum()
# pos = emb[lbls[:, lbl]]
# proj = pos.T @ pos / lbls[:, lbl].sum()
# proj.shape
projs = []
for lbl in range(lbls.shape[1]):
    pos = valemb[lbls[:, lbl]]
    proj = pos.T @ pos / lbls[:, lbl].sum()
    projs.append(proj)
projs = torch.stack(projs)
projs.shape

torch.Size([80, 128, 128])

In [16]:

z = model.encode(batch['image_features'])
y = batch['label']
score = model.predict(z)
# yp = score.sub(thrs.unsqueeze(0)).ge(0)
score.shape

ic = 2
y[:,ic].float().mean()

dz = z @ projs[ic]
dz = z + (-1) ** y[:,ic].unsqueeze(-1) * dz
dz = dz / dz.norm(dim=-1, keepdim=True)
dz.shape 

torch.Size([5000, 80])

In [45]:
obs_results = []
intv_results = []

obs_score = model.predict(z).cpu().numpy()

for ic in tqdm(range(y.shape[1])):
    dz = z @ projs[ic]
    dz = z + (-1) ** y[:,ic].unsqueeze(-1) * dz
    dz = dz / dz.norm(dim=-1, keepdim=True)
    intv_score = model.predict(dz).cpu().numpy()
    obs_results.append(roc_auc_score(gt, obs_score[:, i]))
    intv_row = []
    for i in range(y.shape[1]):
        gt = y[:, i].cpu().numpy()
        if i == ic:
            gt = ~gt
        intv_row.append(roc_auc_score(gt, intv_score[:, i]))
    intv_results.append(intv_row)
obs_results = np.array(obs_results)
intv_results = np.array(intv_results)
obs_results.shape, intv_results.shape

((80,), (80, 80))

In [14]:
model.predict(emb)[idx]

tensor([  9.1324,  -9.0957, -11.5980, -13.6753, -15.8401, -17.5224, -11.4130,
        -10.6889,  -9.6972, -12.5531, -19.4821, -16.7362, -18.4169,  -8.7252,
        -11.3438,  -8.1346,  -8.7017,  -8.9816, -15.1360, -16.9146, -14.9189,
        -13.8886, -17.9677, -13.7450,  -7.1458, -11.8310,  -5.7632,  -7.5476,
        -10.2915, -14.5164, -22.3432, -23.1013, -12.3706, -22.9805, -13.9289,
        -18.5452, -22.8026, -20.0341, -11.0255,  -0.3999,  -6.2802,  -0.1928,
         -8.1223,  -6.1516,  -6.1214,  -2.8473, -10.8983,  -6.5904, -11.2488,
         -7.8607, -17.4817, -12.2376, -13.9882, -16.8319, -15.4528, -13.8059,
         -5.0010,  -7.1793,   0.6025,  -6.3523,  -8.7216,  -0.9808,  -8.0420,
         -6.2589, -13.6159,  -7.4622, -11.5423,  -1.2052,  -9.2549,  -9.9871,
        -12.3797,   5.3217, -10.4644,  -2.8025,  -4.4475,  -0.2001,  -5.8738,
         -7.3922,  -3.4281,  -1.3222], device='cuda:0')

In [15]:
new = (projs[-4] - projs[-5]) @ emb[idx]
new = new / new.norm()
new.shape

torch.Size([128])

In [16]:
model.predict(new.unsqueeze(0))

tensor([[  3.9024, -10.6718,  -6.7774, -13.5819,  -5.4185,  -5.7317, -10.2014,
          -8.8080, -14.0354,  -9.1711, -16.1837,  -8.8150, -13.6464, -10.6974,
          -6.5013, -16.7903, -10.8953, -11.2102, -15.7333,  -9.0341, -13.8807,
         -17.1458, -19.8320, -17.6132,  -6.9916, -12.8268,  -8.4250, -12.4215,
         -12.9349,  -9.5340, -12.5550, -14.9554, -10.9486,  -7.8032, -15.0483,
         -12.4928, -14.6519, -15.7950, -19.9726,  -8.9246, -21.3170, -14.5983,
          -9.4339,  -6.3230, -10.8002,  -8.1817, -10.1597, -11.4045,  -9.9101,
         -14.0311, -13.2979,  -7.0450, -10.4698, -13.5920, -16.6083,  -6.0683,
         -13.4473, -19.2462, -16.3298, -23.2252, -12.8410, -21.5716, -14.6681,
         -14.6640, -12.4886, -14.2852, -11.2389,  -8.4424, -14.2416, -15.9075,
          -8.9048, -10.5014, -15.6897, -11.2378, -14.1076, -27.9013,   0.8805,
         -14.6172, -19.6178, -15.6648]], device='cuda:0')

In [17]:
model.predict(emb).shape

torch.Size([4135, 80])

In [19]:
pred = model.predict(emb)
i = 0
thrs = []
aucs = []
for i in range(pred.shape[1]):
    fpr, tpr, thresholds = roc_curve(lbls[:,i].cpu().numpy(), pred[:,i].cpu().numpy())
    auc = roc_auc_score(lbls[:,i].cpu().numpy(), pred[:,i].cpu().numpy())
    thr = thresholds[(tpr-fpr).argmax()]
    thrs.append(thr)
    aucs.append(auc)
thrs = torch.as_tensor(thrs).to(device)
vaucs = torch.as_tensor(aucs).to(device)
vaucs.mean(), thrs.mean()

(tensor(0.9789, device='cuda:0', dtype=torch.float64),
 tensor(-3.2697, device='cuda:0'))

In [58]:
z = model.encode(batch['image_features'])
y = batch['label']
score = model.predict(z)
yp = score.sub(thrs.unsqueeze(0)).ge(0)
score.shape

torch.Size([5000, 80])

In [54]:

aucs = []
for i in range(pred.shape[1]):
    auc = roc_auc_score(y[:,i].cpu().numpy(), score[:,i].cpu().numpy())
    aucs.append(auc)
aucs = torch.as_tensor(aucs).to(device)
aucs.mean()

tensor(0.9674, device='cuda:2', dtype=torch.float64)

In [59]:
f1_score(y[:,idx].cpu().numpy(), yp[:,idx].cpu().numpy())

np.float64(0.2764505119453925)

In [65]:
outs = []
for idx in range(y.shape[1]):
    p, r, f, _ = precision_recall_fscore_support(y[:,idx].cpu().numpy(), yp[:,idx].cpu().numpy())
    outs.append((p[1], r[1], f[1]))
outs = torch.as_tensor(outs).to(device)
outs.shape

torch.Size([80, 3])

In [68]:
outs[:, -1]#.mean()

tensor([0.9073, 0.2378, 0.5214, 0.5544, 0.8257, 0.5926, 0.6576, 0.4215, 0.6176,
        0.3743, 0.2765, 0.1743, 0.1308, 0.2748, 0.2030, 0.7054, 0.3593, 0.5011,
        0.6087, 0.4420, 0.9438, 0.7500, 0.9390, 0.8919, 0.3023, 0.3384, 0.3568,
        0.4231, 0.2056, 0.4862, 0.8679, 0.4653, 0.5704, 0.7330, 0.6690, 0.8597,
        0.6244, 0.8581, 0.9174, 0.4437, 0.3298, 0.4681, 0.3543, 0.4094, 0.3725,
        0.4373, 0.4373, 0.2147, 0.4462, 0.2431, 0.4769, 0.4409, 0.2479, 0.7949,
        0.3661, 0.4580, 0.5067, 0.3570, 0.1854, 0.4668, 0.5943, 0.6712, 0.5326,
        0.5668, 0.6884, 0.5345, 0.4138, 0.3149, 0.1889, 0.4283, 0.2286, 0.6654,
        0.4093, 0.3803, 0.2857, 0.3659, 0.1366, 0.4501, 0.0613, 0.1299],
       device='cuda:2', dtype=torch.float64)

In [63]:
outs[0]

(array([0.86018117, 0.95002032]),
 array([0.94668401, 0.86817675]),
 array([0.90136195, 0.9072565 ]),
 array([2307, 2693]))